In [1]:
import cv2
import os

def extraer_fotogramas(video_path, output_folder, salto_fotogramas=15):

    # Obtener el nombre del video sin extensión
    base_name = os.path.splitext(os.path.basename(video_path))[0]
    
    cap = cv2.VideoCapture(video_path)
    frame_index = 0  # Contador de fotogramas
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Guardar solo si el fotograma es múltiplo de salto_fotogramas
        if frame_index % salto_fotogramas == 0:
            output_filename = f"{base_name}_fotograma{frame_index}.jpeg"
            output_path = os.path.join(output_folder, output_filename)
            cv2.imwrite(output_path, frame)
        
        frame_index += 1
        
    cap.release()

def procesar_carpeta_videos(input_folder, output_folder, salto_fotogramas=15):
    """
    Recorre todos los videos de la carpeta de entrada y extrae fotogramas,
    guardándolos en la misma carpeta de salida con un nombre único.
    """
    
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Definir las extensiones de video que se desean procesar
    extensiones_video = ('.mp4', '.avi', '.mov', '.mkv')
    
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(extensiones_video):
            video_path = os.path.join(input_folder, filename)
            extraer_fotogramas(video_path, output_folder, salto_fotogramas)


carpeta_entrada = r"E:/Futsal/transform/raw_video/"          # Carpeta que contiene todos los videos
carpeta_salida = r"E:/Futsal/transform/raw_img/"       # Carpeta donde se creará una subcarpeta por cada video

procesar_carpeta_videos(carpeta_entrada, carpeta_salida, salto_fotogramas=5)



In [1]:
import os
from PIL import Image

def preparar_imagenes(input_folder, output_folder, size=(640, 640)):
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Recorrer todos los archivos de la carpeta de entrada
    for filename in os.listdir(input_folder):
        # Filtrar para que procese solo los formatos de imagen deseados
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)
            
            # Abrir la imagen
            with Image.open(input_path) as img:
                # Convertir a blanco y negro
                img_bn = img.convert('L')
                # Redimensionar a 640x640
                img_redimensionada = img_bn.resize(size)
                # Guardar la imagen resultante en la carpeta de salida
                img_redimensionada.save(output_path)


carpeta_entrada = r"E:/Futsal/transform/raw_img"            
carpeta_salida = r"E:/Futsal/transform/img"  
preparar_imagenes(carpeta_entrada, carpeta_salida)


In [ ]:
import os
import shutil
import random

def get_files_by_label(label, labels_dir, images_dir, num_samples):
    matching_files = []
    for label_file in os.listdir(labels_dir):
        label_path = os.path.join(labels_dir, label_file)
        with open(label_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                if line.startswith(str(label)):
                    matching_files.append(label_file.replace('.txt', '.jpg'))  # Assuming images are in JPG format
                    break
    return random.sample(matching_files, min(num_samples, len(matching_files)))

def copy_files(file_list, src_images, src_labels, dest_images, dest_labels):
    os.makedirs(dest_images, exist_ok=True)
    os.makedirs(dest_labels, exist_ok=True)
    for file in file_list:
        image_src = os.path.join(src_images, file)
        label_src = os.path.join(src_labels, file.replace('.jpg', '.txt'))
        image_dest = os.path.join(dest_images, file)
        label_dest = os.path.join(dest_labels, file.replace('.jpg', '.txt'))
        if os.path.exists(image_src) and os.path.exists(label_src):
            shutil.copy(image_src, image_dest)
            shutil.copy(label_src, label_dest)

def main():
    base_dir = '../dataset/train'
    images_dir = os.path.join(base_dir, 'images')
    labels_dir = os.path.join(base_dir, 'labels')
    output_base = '../sampled_dataset/train'
    output_images = os.path.join(output_base, 'images')
    output_labels = os.path.join(output_base, 'labels')
    
    sampled_0 = get_files_by_label(0, labels_dir, images_dir, 100)
    sampled_1 = get_files_by_label(1, labels_dir, images_dir, 100)
    
    copy_files(sampled_0, images_dir, labels_dir, output_images, output_labels)
    copy_files(sampled_1, images_dir, labels_dir, output_images, output_labels)
    
    print(f"Copied {len(sampled_0)} images with label 0 and {len(sampled_1)} images with label 1")

In [2]:
main()

Copied 100 images with label 0 and 100 images with label 1
